please please please be the final version

# import stuff

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
import json
import os
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, ConfusionMatrixDisplay
from hpsklearn import svc
from sklearn.model_selection import train_test_split
from hyperopt import hp

# prep data

In [2]:
# initial dataset stuff
nlp = spacy.load("spacy-twitter") # out of function so you don't load it every time (it takes a while)

# function for glove embeddings
def embed_dataset(dataset_text):
    encoded = np.array([nlp(text).vector for text in dataset_text])
    return encoded.tolist()

# function to load dataset from folder. Also embeds the text.
def get_dataset(name):
    """
    loads a dataset and embeds the text. text must be in column named "text".
    datasets are in the folder datasets/
    name must be a string that's matches the csv file in datasets
    """
    dataset = pd.read_csv(f'datasets\\{name}.csv')
    dataset.rename(columns = {"Unnamed: 0":"entry"}, inplace=True) #the entry label never carries over
    dataset['e_text'] = embed_dataset(dataset['text'])
    return dataset

# train models

In [3]:
from hpsklearn import HyperoptEstimator

#function to optimize model with hyperopt
def optimize_model(model, X_train, y_train, trials):
    """
    optimizes a model with hyperopt. returns the optimized and fitted model.
    
    model: should match a sklearn model - list on github. can preset variables or set a range with hp
    X_train and y_train: training set
    trials: number of trials to do
    """
    mod = HyperoptEstimator(classifier=model,
                            preprocessing=[],
                            max_evals=trials,
                            trial_timeout=120,
                            verbose=False)
    mod.fit(X_train, y_train, random_state=42)
    #print(mod.best_model())
    return mod

def evaluate_model(model, X_test, y_test):
    pred_y = model.predict(X_test)
    acc_mod = accuracy_score(y_test, pred_y)
    print("Accuracy:", float("{0:.2f}".format(acc_mod*100)), "%")
    f1_mod = f1_score(y_test, pred_y, average="macro")
    print("F1:", float("{0:.2f}".format(f1_mod*100)), "%")
    cm = confusion_matrix(y_test, pred_y)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["true", "false"])
    disp.plot()
    plt.show() 

In [4]:
# get breakdown of categories with confidence level, as well as dataset coverage
def get_category_info(dataset_name, dataset, confidence, size_limit):
    file_name = f"{dataset_name}_cats\\{dataset_name}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    info = []
    dataset_coverage = np.zeros([len(dataset)], dtype=bool)
    for category in data.keys():
        cat_entries = [int(i) for i in data[category].keys() if data[category][i] > confidence]
        if len(cat_entries) < size_limit:
            print(f"Skipped category: {category} due to low numbers")
            continue
        info.append((category, len(cat_entries)))
        for entry in cat_entries:
            if dataset_coverage[entry] == False:
                dataset_coverage[entry] = True
    uniques, counts = np.unique(dataset_coverage, return_counts=True)
    percentages = dict(zip(uniques, counts * 100 / len(dataset_coverage)))
    return info, percentages

In [5]:
def train_models(dataset_name, train_set, confidence, size_limit, model_list):
    """
    trains a set of models in each category. returns the best model for each category, in the form {'category': [modelscore, modelname, fittedmodel]}

    dataset_name: a string with the name of the training set. used for calling the category file
    train_set: the training set to use
    confidence: the confidence required to consider an entry part of a category
    size_limit: the number of entries needed in a category to consider that category for training
    model_list: the list of models to train. in the form [("model_name1", model1), ("model_name2", model2), etc]
    """
    file_name = f"{dataset_name}_cats\\{dataset_name}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    f.close()
    category_models = {} #this will be returned
    for category in data.keys(): 
        cat_entries = [int(i) for i in data[category].keys() if data[category][i] > confidence]
        
        # skip category if size of category is below limit
        if len(cat_entries) < size_limit:
            print(f"Skipped category: {category} due to low numbers")
            continue
        
        category_data = train_set.filter(axis=0, items=cat_entries)

        #split validation set
        X = category_data.drop('target', axis=1)
        y = category_data["target"]
        X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

        # skip category if split only has one class
        if (len(np.unique(y_train)) <= 1):
            print(f"Skipped category: {category} due to class issues")
            continue

        X_train_text = np.array([text for text in X_train['e_text']])

        trained_models = []
        # train models from list
        for model_name, model in model_list:
            try:
                optimized = optimize_model(model, X_train_text, y_train, 1)
                best_model = optimized.best_model()['learner'].fit(X_train_text, y_train)
                trained_models.append((model_name, best_model))
            except:
                print(f"Error training {model_name} in category {category}, skipping")
                continue
        
        #get the best model
        X_val_text = np.array([text for text in X_val['e_text']])
        best_model = [0, "x", "x"]
        for model_name, model in trained_models:
            score = model.score(X_val_text, y_val)
            if score > best_model[0]:
                best_model = [score, model_name, model]

        #add best model to list
        category_models[category] = best_model
    return category_models

# voting

In [13]:
def predict_points(trained_models, test_cat_file, X_test):
    """
    predict points using the trained models. returns an array of the predictions

    trained_models: the models trained in each category, in the form {category: [modelscore, modelname, fittedmodel]}
    test_cat_file: the filepath to the organised category file
    X_test: the test set X values
    """
    #load category data
    f = open(test_cat_file)
    category_data = json.load(f)
    f.close()

    # return arrays
    final_predictions = []
    
    #embedded_text = np.array([text for text in X_test['e_text']])
    for index, row in X_test.iterrows():
        test_point = np.array([row['e_text']])
        point_categories = category_data[str(index)]

        # get weights of each point's topic
        topic_weights = {}
        for category in point_categories:
            main_category = category.split("/")[1]
            if main_category not in trained_models.keys():
                continue
            if main_category in topic_weights:
                topic_weights[main_category] += point_categories[category]
            else:
                topic_weights[main_category] = point_categories[category]
        
        #make topic predictions
        model_predictions = []
        for category in topic_weights.keys():
            modelscore, modelname, model = trained_models[category]
            prediction = model.predict(test_point)
            model_predictions.append((prediction[0], modelscore, modelname, category))

        # aggregate predictions
        truefalse_scores = {True: 0, False:0}
        for prediction, modelscore, modelname, category in model_predictions:
            truefalse_scores[prediction] += modelscore
        
        #determine final prediction
        if truefalse_scores[True] > truefalse_scores[False]:
            final_predictions.append(True)
        else:
            final_predictions.append(False)
    return final_predictions

# evaluate results

In [31]:
def check_score(test, pred):
    acc = accuracy_score(test, pred)
    f1 = f1_score(test, pred, average="macro")
    #cm = confusion_matrix(test, pred)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["true", "false"])
    #disp.plot()
    plt.show() 
    return float("{0:.2f}".format(acc*100)), float("{0:.2f}".format(f1*100))

# pipeline

get dataset -> train models on dataset -> make predictions and vote -> evaluate results

In [34]:
def train_and_evaluate(train_set, test_set, confidence, size_limit, model_list):
    """
    Train and evaluate dataset on test set. Returns results and models
    train_set: training set. In the form ["dataset_name", file_reference_name, dataset]
    test_set: testing set(s). In the form [("dataset_name1", file_reference_name1, dataset1), ("dataset_name2", file_reference_name2, dataset2), etc]
    confidence: confidence threshold to consider a training point
    size_limit: size threshold to consider training a category
    model_list: list of models to train, in the form [("model_name1", model1), ("model_name2", model2), etc]
    """
    #prep dataset
    dataset_name = train_set[0]
    ref_name = train_set[1]
    dataset = train_set[2]
    X = dataset.drop("target", axis=1)
    y = dataset["target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
    training_set = pd.concat([X_train, y_train], axis=1)

    #train models
    trained_models = train_models(ref_name, training_set, confidence, size_limit, model_list)

    #make predictions on test set
    test_cat_file = f"{ref_name}_categories.json"
    predictions = predict_points(trained_models, test_cat_file, X_test)
    results = check_score(y_test, predictions)

    final_results = []
    final_results.append((dataset_name, results))

    #make predictions on out of domain test sets
    for set_name, ref_name, set_data in test_set:
        test_cat_file = f"{ref_name}_categories.json"
        X = set_data.drop("target", axis=1)
        y = set_data["target"]
        predictions = predict_points(trained_models, test_cat_file, set_data)
        results = check_score(y, predictions)
        final_results.append((set_name, results))
    
    return trained_models, final_results

In [38]:
def run_tests(tests, confidence, size_limit, model_list):
    test_results = []
    trained_models = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        models, results = train_and_evaluate(train, t, confidence, size_limit, model_list)
        test_results.append((train[0], results))
        trained_models.append((train[0], models))
    return trained_models, test_results

# Prep Models

In [54]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from sklearn.ensemble import RandomForestClassifier

pheme = get_dataset("pheme")
X = pheme.drop("target", axis=1)
y = pheme["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
X_train_text = np.array([text for text in X_train['e_text']])
X_test_text = np.array([text for text in X_test['e_text']])

def objective(n_estimators):
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_features='sqrt',
                                   random_state=42)
    model.fit(X_train_text, y_train)
    y_pred = model.predict(X_test_text)
    accuracy = accuracy_score(y_test, y_pred)
    return {'loss': -accuracy, 'status': STATUS_OK}

search_space = hp.randint('n_estimators', 200, 1000)
algorithm=tpe.suggest
best_params = fmin(
  fn=objective,
  space=search_space,
  algo=algorithm,
  max_evals=20)

 45%|████▌     | 9/20 [13:46<18:53, 103.05s/trial, best loss: -0.8140077821011673]

# run pipelines

In [ ]:
from hpsklearn import svc, k_neighbors_classifier, logistic_regression
model_list = [
    ("SVC", svc("SVC", random_state=42, probability=True)),
    ("KNN", k_neighbors_classifier("knn")),
    ("Logistic Regression", logistic_regression("LR", random_state=42, solver="saga", penalty=hp.choice("penalty", {None, "l1", "l2"}))),]

In [45]:
pheme = get_dataset("pheme")
twitter = get_dataset("twitter")
twitter15 = twitter.iloc[:1491]
twitter16 = twitter.iloc[1491:]
weibo = get_dataset("weibo")
weibo = weibo.drop([1933, 3564])

In [40]:
tests = [["PHEME", "pheme", pheme], ["twitter15", "twitter", twitter15], ["twitter16", "twitter", twitter16]]

In [46]:
tests2 = [["PHEME", "pheme", pheme], ["twitterFULL", "twitter", twitter], ["WEIBO", "weibo", weibo]]

In [35]:
models, results = train_and_evaluate(["PHEME", "pheme", pheme], [["twitter15", "twitter", twitter15], ["twitter16", "twitter", twitter16]], 0.2, 200, model_list)

100%|██████████| 1/1 [00:02<00:00,  2.42s/trial, best loss: 0.2662538699690402]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 1/1 [00:01<00:00,  1.86s/trial, best loss: 0.20740740740740737]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


100%|██████████| 1/1 [00:02<00:00,  2.11s/trial, best loss: 0.1785714285714286]
Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
100%|██████████| 1/1 [00:01<00:00,  1.79s/trial, best loss: 0.2564102564102564]
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Ad

In [47]:
models, results = run_tests(tests2, 0.2, 200, model_list)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.75s/trial, best loss: 0.1785714285714286]
Skipped category: Books & Literature due to low numbers
Skipped category: Reference due to low numbers
Skipped category: Jobs & Education due to low numbers
Skipped category: Health due to low numbers
Skipped category: Business & Industrial due to low numbers
Skipped category: Autos & Vehicles due to low numbers
Skipped category: Food & Drink due to low numbers
100%|██████████| 1/1 [00:01<00:00,  1.73s/trial, best loss: 0.28205128205128205]
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Games due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Sports due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: A

 33%|███▎      | 1/3 [01:20<02:41, 80.74s/it]

100%|██████████| 1/1 [00:01<00:00,  1.96s/trial, best loss: 0.23728813559322037]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


...

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Skipped category: Science due to low numbers



c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 1/1 [00:01<00:00,  1.76s/trial, best loss: 0.2792792792792793]
Skipped category: Shopping due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Real Estate due to low numbers
Skipped category: Jobs & Education due to low numbers
100%|██████████| 1/1 [00:01<00:00,  1.77s/trial, best loss: 0.24285714285714288]
Skipped category: Business & Industrial due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Hobbies & Leisure due to low numbers
Skipped category: Books & Literature due to low numbers
Skipped category: Beauty & Fitness due to low numbers
Skipped category: Pets & Animals due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Autos & Vehicles due to low numbers


100%|██████████| 3/3 [03:13<00:00, 64.64s/it]


In [52]:
results

[('PHEME',
  [('PHEME', (81.4, 79.98)),
   ('twitterFULL', (55.24, 52.99)),
   ('WEIBO', (50.0, 37.38))]),
 ('twitterFULL',
  [('twitterFULL', (73.59, 73.56)),
   ('PHEME', (54.72, 54.52)),
   ('WEIBO', (50.39, 50.35))]),
 ('WEIBO',
  [('WEIBO', (73.74, 73.55)),
   ('PHEME', (51.11, 48.08)),
   ('twitterFULL', (53.55, 53.29))])]

In [ ]:
def organise_results(list_of_results):
    for results in list_of_results:
        for training_set, metrics in results:
            